In [1]:
from config import root
import os
import subprocess as sp
import pandas as pd

In [2]:
source = os.path.join(root, "data", "in_analysis", "fs_subjects", "sub_001", "surf")
outdir = os.path.join(root, "data", "misc")
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [3]:
def asc2obj(path, suffix=""):
    # Header
    with open(path, "r") as f:
        comment = f.readline()
        nV, nF = f.readline().split(" ")
    
    # Data 
    data = pd.read_csv(path, delim_whitespace=True, skiprows=2, header=None)
    data = data.drop(3, axis=1)
    verts = data.loc[0:int(nV)-1, :]
    verts = verts.assign(c="v")
    faces = data.loc[int(nV):int(nV)+int(nF), :].astype(int) + 1
    faces = faces.assign(c="f")
    dir_, fname = os.path.split(path)
    name, ext = os.path.splitext(fname)
    out_path = os.path.join(dir_, name+suffix+".obj")
    
    # Create file
    open(out_path, "w").close()
    
    # Append to file
    with open(out_path, "a") as f:
        verts[["c", 0, 1, 2]].round(6).to_csv(f, sep=" ", index=False, header=False)
        faces[["c", 0, 1, 2]].to_csv(f, sep=" ", index=False, header=False)
    
    return out_path

def srf2obj(surf, hemi, source="./", outdir="./", suffix=""):
    
    cmd = """
    mris_convert {input} {output1} \
        && mv {output1} {output2}
    """
    
    fname = "%s.%s" % (hemi, surf)
    input_ = os.path.join(source, fname)
    output1 = os.path.join(outdir, "%s.%s.asc" % (hemi, surf))
    output2 = os.path.join(outdir, "%s.%s.srf" % (hemi, surf))
    output3 = os.path.join(outdir, "%s.%s.obj" % (hemi, surf))
    
    fcmd = cmd.format(**{
        "input": input_,
        "output1": output1,
        "output2": output2,    
    })
               
    sp.call(fcmd, shell=True)
    asc2obj(output2, suffix=suffix)

    return output3

In [4]:
srf2obj("pial", "lh", source=source, outdir=outdir, suffix="")

'/data/ext/PhD/Studies/PrimingDecay/data/misc/lh.pial.obj'

In [5]:
srf2obj("pial", "rh", source=source, outdir=outdir, suffix="")

'/data/ext/PhD/Studies/PrimingDecay/data/misc/rh.pial.obj'